sgdclassifier | bag of sites | 0.83341

In [260]:
# from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
# from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

from scipy.sparse import hstack

In [3]:
# !unzip capstone_user_identification.zip

### Обработка входных данных

In [106]:
# PATH_TO_DATA = '../capstone_user_identification'
PATH_TO_DATA = '../input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

times = ['time%s' % i for i in range(1, 11)]
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id', parse_dates=times)
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [107]:
train_df.sort_values(by='time1', inplace=True)

In [126]:
pd.crosstab([morning, day, evening, night], y, rownames=['morning', 'day', 'evening', 'night'])

target                          0     1
morning day evening night              
0       0   1       0        7251     0
        1   0       0      122129  2256
1       0   0       0      121884    41

In [6]:
train_test_df = pd.concat([train_df, test_df])
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [8]:
!mkdir tmp

In [9]:
train_test_df_sites[:253561].to_csv('tmp/train.csv', header=False, index=False, sep=' ')
train_test_df_sites[253561:].to_csv('tmp/test.csv', header=False, index=False, sep=' ')

In [311]:
%%time
vectorizer = CountVectorizer(ngram_range=(1, 2))
# vectorizer = TfidfVectorizer(ngram_range=(1, 5), max_features=5000)
with open('tmp/train.csv') as train:
    X_train_sparse = vectorizer.fit_transform(train)
with open('tmp/test.csv') as test:
    X_test_sparse = vectorizer.transform(test)
y = train_df['target']
print(X_train_sparse.shape)

(253561, 358456)
CPU times: user 9.15 s, sys: 47.9 ms, total: 9.2 s
Wall time: 9.2 s


In [261]:
scaler = StandardScaler()
duration_train = (train_df[times].max(axis=1) - train_df[times].min(axis=1))\
                    .astype('timedelta64[ms]').astype(int).values.reshape(-1, 1)
scaler.fit(duration_train)

StandardScaler()

In [473]:
def features(df):
    hour = df['time1'].apply(lambda ts: ts.hour).values
    morning = ((hour >= 7) & (hour <= 11)).astype('int').reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').reshape(-1, 1)
    night = ((hour >= 0) & (hour <= 6)).astype('int').reshape(-1, 1)
    weekday = df.time1.apply(lambda ts: ts.weekday()).values.reshape(-1, 1)
    month = df.time1.apply(lambda ts: ts.month).values.reshape(-1, 1)
    duration = (df[times].max(axis=1) - df[times].min(axis=1))\
                    .astype('timedelta64[ms]').astype(int).values.reshape(-1, 1)
    duration = scaler.transform(duration)
    year_month = df['time1'].apply(lambda t: 100 * t.year + t.month).values.reshape(-1, 1) / 1e5
#     is_weekend = np.isin(weekday, [6])
    is_sunday = np.isin(weekday, [6])
#     weekdays = pd.get_dummies(weekday.flatten()).to_numpy().reshape(7, -1, 1)
    
    # morning, day, evening, night, weekday
    
    return [morning, day, evening, night, is_sunday, duration, year_month]

In [474]:
%%time
X_train_features = features(train_df)
X_test_features = features(test_df)

CPU times: user 9.34 s, sys: 3.26 ms, total: 9.35 s
Wall time: 9.35 s


In [475]:
X_train = hstack([X_train_sparse, *X_train_features])
X_test = hstack([X_test_sparse, *X_test_features])

### Обучение модели

In [281]:
model = SGDClassifier(loss='log', random_state=17, n_jobs=-1)

In [476]:
%%time
time_split = TimeSeriesSplit(n_splits=10)

cv_scores = cross_val_score(model, X_train, y, cv=time_split, scoring='roc_auc', n_jobs=-1)

CPU times: user 219 ms, sys: 42 ms, total: 261 ms
Wall time: 2.19 s


In [477]:
cv_scores, cv_scores.mean(), cv_scores.std()

(array([0.89081737, 0.7568049 , 0.93033489, 0.97098259, 0.90846714,
        0.94892953, 0.94532919, 0.93133649, 0.95697302, 0.94553447]),
 0.9185509591791876,
 0.05824802684461048)

0.9174855435458449,
 0.06680157906314027

In [289]:
model.fit(X_train, y)
logit_test_pred_proba = model.predict_proba(X_test)

In [88]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                               
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [290]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'result.csv')